In [1]:
import mocalum as mc
import numpy as np

koshava_xyz = np.array([500, 500, 0])
no_scans = 1000
# wind field setup
ref_height = 100      # power law reference height
meas_height = 100 
shear_exponent = 0.2  # power law shear exponent
wind_speed = 8       # wind speed at referenec height
wind_dir = 20 # wind direction
w = 0 

# beam steering setup
distance = 1000   # meter
elevation = np.degrees(np.arcsin(meas_height / distance))
angular_res = 1   # degree
azimuth_mid = 45  # central azimuth angle
sector_size = 30  # degree
scan_speed = 1    # degree.s^-1
max_speed = 50    # degree.s^-1
max_acc = 100     # degree.s^-2

# Uncertainty terms
no_sim = 10000 # number of simulations
corr_coef = 0  # correlation coefficient
mu = 0         # we assume no systematic uncertainty
azim_std = 0.1 # degrees
elev_std = 0.1 # degrees
dis_std = 10   # meters
rad_std = 0.1  # m.s-1  In [23]:  
pl_mc = mc.Mocalum()
pl_mc.add_lidar('sterenn', koshava_xyz)
pl_mc.add_lidar('koshava', koshava_xyz)
pl_mc.set_ivap_probing('sterenn', sector_size*2, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)
pl_mc.set_ivap_probing('koshava', sector_size*2, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)

In [41]:
np.where(pl_mc.data.probing.lidar_id.values == 'koshava')

(array([0]),)

In [12]:
pl_mc.x

<xarray.DataArray (time: 61000)>
array([ 757.52169836,  774.25570624,  790.90617316, ..., 1451.51121823,
       1456.44331123, 1461.08406233])
Coordinates:
    lidar_id  <U7 'koshava'
  * time      (time) float64 0.0 1.0 2.0 3.0 ... 6.17e+04 6.17e+04 6.17e+04

In [31]:
pl_mc.data.probing['y'].sel(lidar_id='koshava').values =  pl_mc.x.values

In [35]:
pl_mc.data.probing['y'][1]

<xarray.DataArray 'y' (time: 61000)>
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
    lidar_id  <U7 'sterenn'
  * time      (time) float64 0.0 1.0 2.0 3.0 ... 6.17e+04 6.17e+04 6.17e+04
Attributes:
    long_name:      Y coordinate
    standard_name:  projection_y_coordinate
    units:          m
    URI:

In [17]:
merged_ds = temp.merge(temp2)

In [36]:
merged_ds

NameError: name 'merged_ds' is not defined

In [27]:
temp2.az.interp(time=np.nan)

KeyError: nan